In [1]:
import pandas as pd

# zlibtrunc

In [2]:
zlibtrunc_results_file = "zlib-trunc/zlibtrunc-02-bits-results/results.jsonl"
raw = pd.read_json(zlibtrunc_results_file, lines=True)
zlib_trunc_df = pd.json_normalize(raw.to_dict(orient="records"), sep=".")

sz3_results_1_file = "sz3/sz3-01-algorithm-chunksize-results/results.jsonl"
raw = pd.read_json(sz3_results_1_file, lines=True)
sz3_df_1 = pd.json_normalize(raw.to_dict(orient="records"), sep=".")

sz3_results_2_file = "sz3/sz3-02-algorithm-error-results/results.jsonl"
raw = pd.read_json(sz3_results_2_file, lines=True)
sz3_df_2 = pd.json_normalize(raw.to_dict(orient="records"), sep=".")

# Combine dataframes
df = pd.concat([sz3_df_1, sz3_df_2, zlib_trunc_df], ignore_index=True)


# Make distortion values NaN for 0 truncated bits
df.loc[df['results.mse'] == 0, ['results.mse']] = float('nan')
df.loc[df['results.ks_statistic'] == 0, ['results.ks_statistic']] = float('nan')
df.loc[df['results.earth_mover_distance'] == 0, ['results.earth_mover_distance']] = float('nan')
df.loc[df['results.jensen_shannon_divergence'] == 0, ['results.jensen_shannon_divergence']] = float('nan')
df

,config.branches,config.chunk_size,config.compressor,config.compressor_config.absErrorBound,config.compressor_config.blockSize,config.compressor_config.cmprAlgo,config.compressor_config.errorBoundMode,config.compressor_config.interpAlgo,config.compressor_config.interpAlpha,config.compressor_config.interpAnchorStride,...,results.ks_statistic,results.mse,results.original_size_bytes,results.psnr,results.rel_error_avg,results.rel_error_max,system.host,system.timestamp,config.compressor_config.compressionLevel,config.compressor_config.truncBits
0,AnalysisJetsAuxDyn.pt,1024,sz3,0.001000,128,0,1,1,1.250000,-1,...,0.000090,2.512150e+03,10635492,87.014755,0.001056,0.087830,Niamh,2026-01-29 16:59:08,NaN,NaN
1,AnalysisJetsAuxDyn.eta,1024,sz3,0.001000,128,0,1,1,1.250000,-1,...,NaN,NaN,10635492,NaN,0.000000,0.000000,Niamh,2026-01-29 16:59:38,NaN,NaN
2,AnalysisJetsAuxDyn.phi,1024,sz3,0.001000,128,0,1,1,1.250000,-1,...,NaN,NaN,10635492,NaN,0.000000,0.000000,Niamh,2026-01-29 17:00:08,NaN,NaN
3,AnalysisElectronsAuxDyn.pt,1024,sz3,0.001000,128,0,1,1,1.250000,-1,...,0.002739,2.371645e+03,55500,78.715790,0.004720,0.199442,Niamh,2026-01-29 17:00:08,NaN,NaN
4,AnalysisElectronsAuxDyn.eta,1024,sz3,0.001000,128,0,1,1,1.250000,-1,...,NaN,NaN,55500,NaN,0.000000,0.000000,Niamh,2026-01-29 17:00:09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,AnalysisJetsAuxDyn.eta,65536,zlib-trunc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.216767,6.790482e-01,10635492,21.351120,0.241489,0.500000,Niamh,2026-01-28 12:24:50,5,23
524,AnalysisJetsAuxDyn.phi,65536,zlib-trunc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.190197,2.764731e-01,10635492,21.547062,0.269270,0.500000,Niamh,2026-01-28 12:25:00,5,23
525,AnalysisElectronsAuxDyn.pt,65536,zlib-trunc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.443243,1.174993e+07,55500,41.765934,0.288008,0.499959,Niamh,2026-01-28 12:25:00,5,23
526,AnalysisElectronsAuxDyn.eta,65536,zlib-trunc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.171892,1.350175e-01,55500,22.662504,0.258806,0.499976,Niamh,2026-01-28 12:25:00,5,23


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
# Plot Compression Ratio vs Quality Metrics
# df = df[df['config.compressor'] == 'sz3']
branches = ['AnalysisJetsAuxDyn.pt', 'AnalysisJetsAuxDyn.eta', 'AnalysisJetsAuxDyn.phi']

# Sort rows by (compression ratio, shannon divergence)
df = df.sort_values(by=['results.compression_ratio', 'results.jensen_shannon_divergence'])

px.scatter(
    df,
    x='results.compression_ratio',
    y='results.jensen_shannon_divergence',
    color='config.compressor_config.cmprAlgo',
    facet_col='config.branches',
    log_y=True
)